In [ ]:
from IPython.display import clear_output
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
from matplotlib.animation import FuncAnimation

def get_stock_data(stock_symbol, interval='1m', hours_back=30):
    # Retrieve the stock data using yfinance
    data = yf.download(stock_symbol, interval=interval, start=datetime.datetime.now()-pd.to_timedelta(hours_back, 'H'), threads=False)
    
    return data

def simple_moving_average_strategy(data):
    # Implement the simple moving average strategy
    signals = pd.DataFrame(index=data.index)
    signals['short_mavg'] = data['Close'].rolling(window=10, min_periods=1, center=False).mean()
    signals['long_mavg'] = data['Close'].rolling(window=30, min_periods=1, center=False).mean()
    signals['positions'] = 0.0
    
    signals['positions'][10:] = np.where(signals['short_mavg'][10:] > signals['long_mavg'][10:], 1.0, 0.0)    # Buy signal
    signals['positions'][:-10] = np.where(signals['short_mavg'][:-10] < signals['long_mavg'][:-10], -1.0, 0.0) # Sell signal
    
    return signals

def update(frame):
    global stock_symbol, signals, data
    
    new_data = get_stock_data(stock_symbol, interval='1m', hours_back=30)
    signals = simple_moving_average_strategy(new_data)

    clear_output(wait=True)
    plt.clf()

    plt.plot(new_data['Close'], label=f'{stock_symbol} Price')

    if not signals.empty:
        plt.plot(signals['short_mavg'], label='Short MA', linestyle='-')
        plt.plot(signals['long_mavg'], label='Long MA', linestyle='-')

        buy_indices = signals.loc[signals.positions == 1.0].index
        buy_values = signals.short_mavg[signals.positions == 1.0]
        plt.scatter(buy_indices, buy_values, marker='^', color='g', label='Buy', alpha=1, s=100)

        sell_indices = signals.loc[signals.positions == -1.0].index
        sell_values = signals.short_mavg[signals.positions == -1.0]
        plt.scatter(sell_indices, sell_values, marker='v', color='r', label='Sell', alpha=1, s=100)

        for index, row in signals.iterrows():
            if row['positions'] == 1.0:
                profit_loss = new_data.loc[index:, 'Close'].pct_change().sum()
                plt.text(index, new_data.loc[index, 'Close'], f'{profit_loss * 100:.2f}%', color='g', fontsize=10, ha='right')
            elif row['positions'] == -1.0:
                profit_loss = new_data.loc[index:, 'Close'].pct_change().sum()
                plt.text(index, new_data.loc[index, 'Close'], f'{profit_loss * 100:.2f}%', color='r', fontsize=10, ha='left')

    plt.title(f'{stock_symbol} - Real-Time Moving Average Strategy')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
